# Object Creation: Notification Objects

This notebook demonstrates how to use the NotificationEventManager class to process notification events and create notification objects from OCED data.

### Notes:
- Update the file paths and user ID to match your data
- The notification objects track the last action (RECEIVED/READ) and its timestamp
- Each notification is linked to both the user and the day it occurred
- The extended data includes both the original OCED data and the new notification objects

## 1. Setup

In [1]:
import datetime
import sys
# Add the project root directory to the Python path
sys.path.append('..')
from concurrent.futures import ThreadPoolExecutor
from src.oced.oced_data_query import OCEDDataQuery
from src.oced.time_objects import TimeObject
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
import gc
import json
import pandas as pd
from pathlib import Path
from typing import Dict, Any
from src.oced.notification_events import NotificationEventManager

## 2. Load OCED Data

In [2]:
# Get OCED-mHealth Data from JSON File
dataQuery = OCEDDataQuery()  
oced_data_file = f"oced_data_time_bouts.json"
data_dict = dataQuery.load_json(oced_data_file)

# Quick look at the data structure
print(f"Number of behavior events: {len(data_dict.get('behaviorEvents', []))}")
print(f"Number of objects: {len(data_dict.get('objects', []))}")

Number of behavior events: 134
Number of objects: 39


In [9]:
# Diagnostic code to inspect notification events
notification_events = [
    notif for notif in data_dict.get('behaviorEvents', [])
    if notif['behaviorEventType'] == 'notification'
]
print(len(notification_events))
notification_events

62


[{'id': 'aacb0577-c58c-4e1e-90da-804a395c95d4',
  'behaviorEventType': 'notification',
  'time': '2025-05-14T20:35:00.814000',
  'behaviorEventTypeAttributes': [{'name': 'action', 'value': 'RECEIVED'}],
  'relationships': [{'id': 'e4365330-3468-48c3-bce3-145668e06f8f',
    'type': 'object',
    'qualifier': 'source'},
   {'id': '3b2a0dba-f13d-4023-8530-c0f0aa8c7b5b',
    'type': 'object',
    'qualifier': 'occurred_on'},
   {'id': '9bdf9992-2d94-426f-a982-5c76a00d333d',
    'type': 'object',
    'qualifier': 'notifies'}],
  'event_type': 'behavior'},
 {'id': 'a5755e86-621e-432f-9ec1-cecfea6545a9',
  'behaviorEventType': 'notification',
  'time': '2025-05-14T17:15:02.688000',
  'behaviorEventTypeAttributes': [{'name': 'action', 'value': 'RECEIVED'}],
  'relationships': [{'id': 'e4365330-3468-48c3-bce3-145668e06f8f',
    'type': 'object',
    'qualifier': 'source'},
   {'id': '3b2a0dba-f13d-4023-8530-c0f0aa8c7b5b',
    'type': 'object',
    'qualifier': 'occurred_on'},
   {'id': '88cf679

## 3. Initialize Notification Manager and Process Data

In [3]:
# Initialize the manager
notification_manager = NotificationEventManager()

# Add notification object type to the data
oced_data_with_notification_objects = notification_manager.create_notification_object_type(data_dict)

# Create notification objects and link events
extended_data, notification_objects = notification_manager.create_notification_objects(
    data=oced_data_with_notification_objects,
    user_id='e4365330-3468-48c3-bce3-145668e06f8f'
)

Notification object type added.

Found 62 notification events
Found 0 existing notification objects


Processing notification events: 100%|██████████| 62/62 [00:00<00:00, 1833.12it/s]

Creating new notification object feba81ba-63c9-474a-a4cf-0c07a80b3fa6 for received event
Linking read event to notification feba81ba-63c9-474a-a4cf-0c07a80b3fa6 (received at 2025-05-09 08:51:37.242000)
Creating new notification object d4d821aa-c478-41c3-95fa-00d3af68b01b for received event
Linking read event to notification d4d821aa-c478-41c3-95fa-00d3af68b01b (received at 2025-05-09 11:55:02.267000)
Creating new notification object 8f394eff-b827-4b06-a006-4560b39a70de for received event
Linking read event to notification 8f394eff-b827-4b06-a006-4560b39a70de (received at 2025-05-09 15:00:37.186000)
Linking read event to notification 8f394eff-b827-4b06-a006-4560b39a70de (received at 2025-05-09 15:00:37.186000)
Creating new notification object a751ef52-e6e8-444e-ae73-56e7ea42c016 for received event
Linking read event to notification a751ef52-e6e8-444e-ae73-56e7ea42c016 (received at 2025-05-09 18:09:19.983000)
Creating new notification object 1eb516b7-d559-49f3-bffe-69ed5e809c75 for recei

## 4. Analyse results

In [4]:
# Print basic statistics
print(f"Total notification objects created: {len(notification_objects)}")

# Count notifications by last action
action_counts = {}
for obj in notification_objects:
    last_action = next(
        (attr['value'] for attr in obj['attributes'] 
         if attr['name'] == 'last_action'),
        None
    )
    action_counts[last_action] = action_counts.get(last_action, 0) + 1

print("\nNotifications by last action:")
for action, count in action_counts.items():
    print(f"- {action}: {count}")

# Optional: Create a DataFrame for more analysis
notifications_df = pd.DataFrame([
    {
        'notification_id': obj['id'],
        'last_action': next((attr['value'] for attr in obj['attributes'] 
                            if attr['name'] == 'last_action'), None),
        'last_action_time': next((attr['time'] for attr in obj['attributes'] 
                                if attr['name'] == 'last_action'), None)
    }
    for obj in notification_objects
])

# Diagnostic code to inspect notification events
day_objects = [
    object for object in data_dict.get('objects', [])
    if object['type'] == 'day'
]
print(len(day_objects))
day_objects

# Display first few notifications
notifications_df.head()



Total notification objects created: 36

Notifications by last action:
- READ: 22
- RECEIVED: 14
24


,notification_id,last_action,last_action_time
0,feba81ba-63c9-474a-a4cf-0c07a80b3fa6,READ,2025-05-09T08:51:55.800000
1,d4d821aa-c478-41c3-95fa-00d3af68b01b,READ,2025-05-09T11:55:11.964000
2,8f394eff-b827-4b06-a006-4560b39a70de,READ,2025-05-09T17:35:18.626000
3,a751ef52-e6e8-444e-ae73-56e7ea42c016,READ,2025-05-09T19:30:36.761000
4,1eb516b7-d559-49f3-bffe-69ed5e809c75,READ,2025-05-10T09:09:33.063000


## 5. Save Processed Data

In [5]:
# Save the extended data
output_file = "notifications_processed.json"  # Update this name as needed
notification_manager.save_extended_data(
    filename=output_file,
    extended_data=extended_data,
    compress=False  # Set to True for compressed output
)


Saving extended data with:
- 36 notification objects
- 62 notification events
- 75 total objects
- 134 total behavior events
Saved extended data to: c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\notebooks\..\data\transformed\notifications_processed.json


## 6. Example: Query Notifications for a Specific Day

In [6]:
# Get notifications for a specific day
date_str = "2025-05-11"  # Update this date
day_notifications = notification_manager.get_notifications_for_day(
    date_str=date_str,
    data=extended_data
)

print(f"\nNotifications for {date_str}:")
for notif in day_notifications:
    last_action = next((attr['value'] for attr in notif['attributes'] 
                       if attr['name'] == 'last_action'), None)
    last_action_time = next((attr['time'] for attr in notif['attributes'] 
                            if attr['name'] == 'last_action'), None)
    print(f"- ID: {notif['id']}, Last Action: {last_action} at {last_action_time}")


Notifications for 2025-05-11:
- ID: 01676469-0c18-435c-8bf1-5c528b75e94a, Last Action: READ at 2025-05-11T08:52:33.183000
- ID: 33430356-421f-4608-9b46-898861de2761, Last Action: RECEIVED at 2025-05-11T11:54:15.745000
- ID: 6b32128f-e1e2-4286-b508-bab6e0688b24, Last Action: RECEIVED at 2025-05-11T11:54:16.238000
- ID: f0c64883-eed5-46c2-abf9-337f66ec62fa, Last Action: READ at 2025-05-11T13:00:00.256000
- ID: d77925fb-0cd4-436d-b4f5-787ff04daaef, Last Action: READ at 2025-05-11T14:55:12.557000
- ID: 9553f0d1-0178-43ca-8c43-0dfbd71247f0, Last Action: RECEIVED at 2025-05-11T17:55:40.027000
- ID: 53ac9df1-e187-4886-87a6-08c3c1bd1c4a, Last Action: RECEIVED at 2025-05-11T17:55:40.758000
- ID: 18c93ce7-c90e-49c7-bc92-61ea60af6e4d, Last Action: READ at 2025-05-11T22:35:53.998000


## 7. Example: Get Specific Notification Details

In [ ]:
# Get details for a specific notification
notification_id = notification_objects[0]['id']  # Example: first notification
notification = notification_manager.get_notification_object(notification_id)

if notification:
    print(f"\nDetails for notification {notification_id}:")
    for attr in notification['attributes']:
        print(f"- {attr['name']}: {attr['value']} (at {attr['time']})")
    print("\nRelationships:")
    for rel in notification['relationships']:
        print(f"- {rel['qualifier']}: {rel['id']} ({rel['type']})")